# Price an FX Forward on broken dates or non standard tenors

This notebook explains how to price an FX Forward on broken dates or non standard tenors.

## Imports

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import fx_forwards
from lseg_analytics.pricing.common import RelativeAdjustableDate, AdjustableDate, ReferenceDate
import pandas as pd
from IPython.display import display

## Create FX forwards using different tenors and dates
To define the FX forward start date and end dates, it is possible to either define them using a broken date, or to use a non standard tenor.
Here is a list of tenors:
- Combination of Days (D), Weeks (W), Months (M) and Years (Y).  Examples: "5D","22W","30Y","2M","1Y5M2D",....
- Overnight tenor. Example: "ON". 
- Tomorrow next tenor. Example: "TN"
- Spot next tenor. Example: "SN"
- Spot week tenor. Example: "SW"
- IMM tenors. Examples: "IMM1","IMM2","IMM3","IMM4"
- Beginning of month tenor. Examples: "JANB","FEBB,""MARB","APRB","MAYB","JUNB","JULB","AUGB","SEPB","OCTB","NOVB","DECB"
- End of month tenors. Examples: "JANM","FEBM,""MARM","APRM","MAYM","JUNM","JULM","AUGM","SEPM","OCTM","NOVM","DECM"
- Central bank meeting dates tenors (Period whose end date is a central bank meeting date). Examples: "FED1","FED2","FED3", "BOE1", "ECB1"

In [2]:
fx_forwards_map = {
    # Create an FX Forward whose start date is spot date, and end date is 3M after the start date.
    "SPOT/3M": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            end_date=RelativeAdjustableDate(
                tenor="3M", reference_date=ReferenceDate.START_DATE
            ),
            payer="Party1",
            receiver="Party2",
        )
    ),
    # Create an FX Forward whose start date is spot date, and end date is the last open day of september after the start date.
    "SPOT/SEPM": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            end_date=RelativeAdjustableDate(
                tenor="SEPM", reference_date=ReferenceDate.START_DATE
            ),
            payer="Party1",
            receiver="Party2",
        )
    ),
    # Create an FX Forward whose start date is spot date, and end date is the first open day of october after the start date.
    "SPOT/OCTM": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            end_date=RelativeAdjustableDate(
                tenor="OCTB", reference_date=ReferenceDate.START_DATE
            ),
            payer="Party1",
            receiver="Party2",
        )
    ),
    # Create an FX Forward whose start date is spot date, and end date is the third Europen Central Bank meeting date after the start date.
    "SPOT/ECB3": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            end_date=RelativeAdjustableDate(
                tenor="ECB3", reference_date=ReferenceDate.START_DATE
            ),
            payer="Party1",
            receiver="Party2",
        )
    ),
    # Create an FX Forward whose start date is spot date, and end date is one month and 15 days after the start date.
    "SPOT/1M15D": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            end_date=RelativeAdjustableDate(
                tenor="1M15D", reference_date=ReferenceDate.START_DATE
            ),
            payer="Party1",
            receiver="Party2",
        )
    ),
    # Create an FX Forward Forward instrument whose start date is 3 months after spot date, and end date is 6 months after the start date
    "3M/6M": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            start_date=RelativeAdjustableDate(
                tenor="3M", reference_date=ReferenceDate.SPOT_DATE
            ),
            end_date=RelativeAdjustableDate(
                tenor="6M", reference_date=ReferenceDate.START_DATE
            ),
            payer="Party1",
            receiver="Party2",
        )
    ),
    # Create an FX Forward Forward instrument whose start date is the January 1rst 2026 (adjusted from holiday), and end date June 1rst 2026 (adjusted from holiday)
    "01JAN2026/06JAN2026": fx_forwards.FxForward(
        definition=fx_forwards.FxForwardDefinition(
            quoted_currency="USD",
            base_currency="EUR",
            start_date=AdjustableDate(date="2026-01-01"),
            end_date=AdjustableDate(date="2026-01-06"),
            payer="Party1",
            receiver="Party2",
        )
    ),
}

## Compute dates for each FX forward
Below we price each FxForward to check which start dates and end dates are used in the calculation.

In [3]:
# Define the valuation date
valuation_date = "2024-02-11"

# Loop through each FX Forward in the map and price it
pricing_results = {}
for key, fx_forward in fx_forwards_map.items():
    fx_forward.save(name="temp_forward", space="HOME")
    pricing_results[key] = fx_forward.price(
        pricing_preferences=fx_forwards.FxPricingParameters(
            valuation_date=valuation_date
        )
    )
    fx_forwards.delete(name="temp_forward", space="HOME")

## Display the response values
Print the description of the response to see the start date, end date, and swap point bid/ask for each Fx Forward

In [4]:
results = [
    {
        "Case": key,
        "StartDate": result.analytics.description.start_date.adjusted,
        "EndDate": result.analytics.description.end_date.adjusted,
        "Swap points (Bid)": result.analytics.pricing_analysis.fx_swaps_ccy1_ccy2.bid,
        "Swap points (Ask)": result.analytics.pricing_analysis.fx_swaps_ccy1_ccy2.ask,
    }
    for key, result in pricing_results.items()
]

df = pd.DataFrame(results)
display(df)

,Case,StartDate,EndDate,Swap points (Bid),Swap points (Ask)
0,SPOT/3M,2024-02-13,2024-05-13,39.770000,40.170000
1,SPOT/SEPM,2024-02-13,2024-09-30,101.616562,104.175625
2,SPOT/OCTM,2024-02-13,2024-10-01,102.088125,104.621250
3,SPOT/ECB3,2024-02-13,2024-06-12,51.799032,53.747419
4,SPOT/1M15D,2024-02-13,2024-03-28,19.156364,19.354545
5,3M/6M,2024-05-13,2024-11-13,81.580000,87.080000
6,01JAN2026/06JAN2026,2026-01-02,2026-01-06,-7.294130,11.149348
